In [1]:
import numpy as np
import shear_util as su

# Calculate moment radius in two ways

In [2]:
Q = np.array([[3  , 0.2],
              [0.2,   4]])

su.calc_moment_radius([Q,2*Q], method='det'), su.calc_moment_radius([Q,2*Q,3*Q], method='trace')

(array([1.85965677, 2.62995182]), array([1.87082869, 2.64575131, 3.24037035]))

# Matrix diagonalization

In [3]:
su.diagonalize(Q)

array([[2.96148352, 0.        ],
       [0.        , 4.03851648]])

In [4]:
# vectorization test
su.diagonalize([Q,Q,2*Q])

array([[[2.96148352, 0.        ],
        [0.        , 4.03851648]],

       [[2.96148352, 0.        ],
        [0.        , 4.03851648]],

       [[5.92296704, 0.        ],
        [0.        , 8.07703296]]])

# Compare two methods of finding hlr from a second-moment tensor

In [5]:
# su.hlr_from_moments(Q) is not as efficient

Q = np.array([[3, 0],
              [0, 3]])

su.hlr_from_moments(Q), su.hlr_from_moments_fast(Q)

(2.0393339803376183, 2.039333980337618)

In [6]:
Q = np.array([[3  , 0.2],
              [0.2,   4]])

su.hlr_from_moments(Q), su.hlr_from_moments_fast(Q)

(2.194162032740561, 2.1941620327329114)

In [7]:
Q = np.array([[2   , -0.61],
              [-0.61,    1]])

su.hlr_from_moments(Q), su.hlr_from_moments_fast([Q,Q])

(1.3718477568917127, array([1.37184776, 1.37184776]))

In [ ]:
su.get_shape_covmat_fast(hlr_array,e1_array,e2)

Let's start with a second-moment matrix, `Q`, and see if we can get the same `Q` back after using the inverse function.

In [8]:
Q = np.array([[2.3, 1.4],
              [1.4, 1.5]])

hlr_Q, e1, e2 = su.hlr_from_moments_fast(Q, return_shape=True)
print('hlr_Q =', hlr_Q)
Q_out = su.get_shape_covmat_fast(hlr_Q,e1,e2)
print('\nQ_out =\n\n', Q_out)
hlr_Q_out = su.hlr_from_moments_fast(Q_out)
print('\nhlr_Q_out =', hlr_Q_out) # should be equal to hlr_Q

hlr_Q = 1.4398168504108615

Q_out =

 [[2.3 1.4]
 [1.4 1.5]]

hlr_Q_out = 1.4398168504108615


In [9]:
su.shape_from_moments([Q,Q], return_emag=False)

(array([0.1281782, 0.1281782]), array([0.44862369, 0.44862369]))

In [10]:
su.shape_from_moments([Q,Q], return_emag=True)

(array([0.1281782, 0.1281782]),
 array([0.44862369, 0.44862369]),
 array([0.46657568, 0.46657568]))

In [11]:
su.hlr_from_moments_fast([Q,Q], return_shape=False)

array([1.43981685, 1.43981685])

In [12]:
su.hlr_from_moments_fast([Q,Q], return_shape=True)

(array([1.43981685, 1.43981685]),
 array([0.1281782, 0.1281782]),
 array([0.44862369, 0.44862369]))

# More vectorization tests

In [13]:
Q_multi = np.array([[[2, 0], [0, 1]],
                    [[1, 0.1], [0.1, 1]],
                    [[3.5, 0.9], [0.9, 4.0]]])

su.hlr_from_moments_fast(Q_multi)

array([1.41508108, 1.17548171, 2.25656126])

In [14]:
e1, e2, e = su.shape_from_moments(Q_multi)
e1, e2, e

(array([ 0.17157288,  0.        , -0.03386706]),
 array([0.        , 0.05012563, 0.12192142]),
 array([0.17157288, 0.05012563, 0.12653778]))

In [15]:
# compare it to WLD estimation
sigma_m,sigma_p,a,b,beta,e1,e2 = su.moments_size_and_shape(Q_multi)
e1, e2, e

(array([ 0.17157288,  0.        , -0.03386706]),
 array([0.        , 0.05012563, 0.12192142]),
 array([0.17157288, 0.05012563, 0.12653778]))

In [16]:
# try on the first matrix one in the Q array
su.shape_from_moments(Q_multi[0])

(0.1715728752538099, 0.0, 0.1715728752538099)

# Test the HLR shear model

In [17]:
hsm = su.HLRShearModel()
hlr_postshear = 0.8 # arcsec
e = 0.44 # ellipticity magnitude
hlr_preshear = hsm.get_hlr_preshear(hlr_postshear, e, magnify=True)
hlr_preshear

0.6584671759264941

In [18]:
hlr_preshear = hsm.get_hlr_preshear(hlr_postshear, e, magnify=False)
hlr_preshear

0.7332614798899851

In [19]:
hlr_preshear = 1.0
e = 0.52
hlr_postshear = hsm.get_hlr_preshear(hlr_preshear, e, magnify=True)
hlr_postshear

0.7451352885166235

In [20]:
hlr_postshear = hsm.get_hlr_preshear(hlr_preshear, e, magnify=False)
hlr_postshear

0.8723539233897909

Let's start with an `hlr` and `e` pair and see if we can get the same `hlr` back by using two consecutive operation that are inverse of eachother.

In [21]:
hlr_preshear = 1.05
e = 0.7
hsm.get_hlr_postshear(hsm.get_hlr_preshear(hlr_preshear, e, magnify=True), e, magnify=True)

1.05

In [22]:
hsm.get_hlr_postshear(hsm.get_hlr_preshear(hlr_preshear, e, magnify=False), e, magnify=False)

1.05

It sholdn't work if you magnify one of them and not the other one (for non-zero `e`).

In [23]:
hsm.get_hlr_postshear(hsm.get_hlr_preshear(hlr_preshear, e, magnify=False), e, magnify=True)

1.4702940843958747

In [24]:
hsm.get_hlr_postshear(hsm.get_hlr_preshear(hlr_preshear, e, magnify=True), e, magnify=False)

0.7498499869521024